# Maps

Hash maps, also known as hash tables, are data structures that store key-value pairs for efficient data retrieval. They use a hash function to compute an index (or hash code) into an array of buckets or slots, from which the desired value can be found. This allows for average-case constant time complexity (`O(1)`) for lookups, insertions, and deletions. Hash maps are widely used in programming for tasks like implementing associative arrays, caching, and managing unique items. However, they can face issues like collisions (when two keys hash to the same index), which are typically handled through techniques like chaining or open addressing.

In [1]:
class HashMap[K, V]:
    def __init__(self, size=5):
        """Initialize the hashmap with a given size."""
        self.size = size
        self.map = [[] for _ in range(size)] # Create a list of empty lists for chaining

    def _hash(self, key: K):
        """Hash function to compute the index for a given key."""
        return hash(key) % self.size

    def insert(self, key: K, value: V):
        """Insert a key-value pair into the hashmap."""
        index = self._hash(key)
        # Check if the key already exists and update it
        for i, (k, v) in enumerate(self.map[index]):
            if k == key:
                self.map[index][i] = (key, value)  # Update existing key
                return
        # If the key does not exist, append a new key-value pair
        self.map[index].append((key, value))

    def get(self, key: K) -> V:
        """Retrieve the value associated with the given key."""
        index = self._hash(key)
        for k, v in self.map[index]:
            if k == key:
                return v  # Return the value if the key is found
        return None  # Return None if the key is not found

    def delete(self, key: K):
        """Delete the key-value pair associated with the given key."""
        index = self._hash(key)
        for i, (k, v) in enumerate(self.map[index]):
            if k == key:
                del self.map[index][i]  # Remove the key-value pair
                return True  # Return True if deletion was successful
        return False  # Return False if the key was not found

    def __repr__(self):
        return str(self)

    def __str__(self):
        """String representation of the hashmap for easy debugging."""
        return str(self.map)

hashmap = HashMap()

hashmap.insert("key1", "value1")
hashmap.insert("key2", "value2")
assert hashmap.get("key1") == "value1"


hashmap.insert("key1", "value3")
assert hashmap.get("key1") == "value3"
assert hashmap.get("key2") == "value2"

hashmap.delete("key1")
assert hashmap.get("key1") is None

hashmap

[[], [], [], [('key2', 'value2')], []]

## LRU

LRU stands for "Least Recently Used," which is a cache management algorithm. It keeps track of the order in which items are accessed and removes the least recently used items first when the cache reaches its limit. This approach helps optimize memory usage by ensuring that frequently accessed data remains available while less frequently accessed data is discarded.

In [2]:
from typing import Dict, Optional


class Node[T]():
    value: T
    next: Optional["Node[T]"] = None
    prev: Optional["Node[T]"] = None

    def __init__(self, value: T):
        self.value = value

class LRU[K, V]():
    length: int = 0
    capacity: int = 0
    head: Node[V] | None = None
    tail: Node[V] | None = None
    lookup: HashMap[K, Node[V]]
    reverse_lookup: HashMap[Node[V], K]

    def __init__(self, capacity: int = 10):
        self.capacity = capacity
        self.lookup = HashMap()
        self.reverse_lookup = HashMap()

    def update(self, key: K, value: V):
        node = self.lookup.get(key)
        if node is None:
            node = Node(value)
            self.length += 1
            self.prepend(node)
            self.trim_cache()

            self.lookup.insert(key, node)
            self.reverse_lookup.insert(node, key)

        else:
            self.detach(node)
            self.prepend(node)
            node.value = value
        

    def get(self, key: K) -> V | None:
        node = self.lookup.get(key)
        if node is None:
            return

        self.detach(node)
        self.prepend(node)

        return node.value
    
    def detach(self, node: Node[V]):
        if node.prev:
            node.prev.next = node.next
        
        if node.next:
            node.next.prev = node.prev

        if self.head == node:
            self.head = self.head.next

        if self.tail == node:
            self.tail = self.tail.prev

        node.next = None
        node.prev = None
        

    def prepend(self, node: Node[V]):
        if self.head is None:
            self.head = self.tail = node
            return
        
        node.next = self.head
        self.head.prev = node
        self.head = node

    def trim_cache(self):
        if self.length <= self.capacity:
            return
        
        tail = self.tail
        self.detach(self.tail)
        key = self.reverse_lookup.get(tail)
        
        self.lookup.delete(key)
        self.reverse_lookup.delete(tail)

        self.length -= 1


lru = LRU(3)

assert lru.get("foo") is None
lru.update("foo", 69)
assert lru.get("foo") == 69

lru.update("bar", 420)
assert lru.get("bar") == 420

lru.update("baz", 1337)
assert lru.get("baz") == 1337

lru.update("ball", 69420)
assert lru.get("ball") == 69420

assert lru.get("foo") is None
assert lru.get("bar") == 420

lru.update("foo", 69)
assert lru.get("bar") == 420
assert lru.get("foo") == 69

assert lru.get("baz") is None